In [3]:
!pip install earthengine-api

In [2]:
!earthengine authenticate

'earthengine' is not recognized as an internal or external command,
operable program or batch file.


MODEL 01

In [ ]:
import datetime
import ee
from ipyleaflet import Map, DrawControl

# Autentificare și inițializare Earth Engine
ee.Initialize()

# Definirea unei funcții pentru extragerea informațiilor SMAP pentru regiunea specificată
def get_smap_info_for_region(geometry):
    smap_bands = ee.ImageCollection("NASA/SMAP/SPL4SMGP/007")
    smap_bands_in_region = smap_bands.filterBounds(geometry)
    first_smap_bands = smap_bands_in_region.first()

    smap_data = first_smap_bands.select(['sm_surface', 'sm_rootzone','surface_temp', 'land_evapotranspiration_flux', 'soil_water_infiltration_flux']).reduceRegion(
        reducer=ee.Reducer.mean(), geometry=geometry, scale=9000).getInfo()

    surface_soil_moisture = smap_data['sm_surface']
    root_zone_soil_moisture = smap_data['sm_rootzone']
    land_zone_evapotranspiration_flux = smap_data['land_evapotranspiration_flux']
    soil_zone_water_infiltration_flux = smap_data['soil_water_infiltration_flux']

    surface_zone_temp_kelvin = smap_data['surface_temp']
    surface_zone_temp_celsius = surface_zone_temp_kelvin - 273.15

    return {
        "surface_soil_moisture": surface_soil_moisture,
        "root_zone_soil_moisture": root_zone_soil_moisture,
        "surface_zone_temp_celsius": surface_zone_temp_celsius,
        "land_zone_evapotranspiration_flux": land_zone_evapotranspiration_flux,
        "soil_zone_water_infiltration_flux": soil_zone_water_infiltration_flux
    }


# Funcția pentru extragerea informațiilor din setul de date ESA WorldCereal pentru regiunea specificată
def get_esa_info_for_region(geometry):
    aez_dataset = ee.FeatureCollection("ESA/WorldCereal/AEZ/v100")
    features = aez_dataset.filterBounds(geometry).getInfo()['features']

    if len(features) > 0:
        properties = features[0]['properties']
        aez_id = properties['aez_id']
        aez_groupid = properties['aez_groupid']

        # Obținerea valorilor pentru zi și lună
        tc_annual_sos_day = properties['tc-annual_sos'] % 100
        tc_annual_sos_month = properties['tc-annual_sos'] // 100
        tc_annual_eos_day = properties['tc-annual_eos'] % 100
        tc_annual_eos_month = properties['tc-annual_eos'] // 100
        tc_wintercereals_sos_day = properties['tc-wintercereals_sos'] % 100
        tc_wintercereals_sos_month = properties['tc-wintercereals_sos'] // 100
        tc_wintercereals_eos_day = properties['tc-wintercereals_eos'] % 100
        tc_wintercereals_eos_month = properties['tc-wintercereals_eos'] // 100
        tc_springcereals_sos_day = properties['tc-springcereals_sos'] % 100
        tc_springcereals_sos_month = properties['tc-springcereals_sos'] // 100
        tc_springcereals_eos_day = properties['tc-springcereals_eos'] % 100
        tc_springcereals_eos_month = properties['tc-springcereals_eos'] // 100

        tc_annual_sos = f"{tc_annual_sos_day} (Ziua) din {tc_annual_sos_month} (Luna)"
        tc_annual_eos = f"{tc_annual_eos_day} (Ziua) din {tc_annual_eos_month} (Luna)"
        tc_wintercereals_sos = f"{tc_wintercereals_sos_day} (Ziua) din {tc_wintercereals_sos_month} (Luna)"
        tc_wintercereals_eos = f"{tc_wintercereals_eos_day} (Ziua) din {tc_wintercereals_eos_month} (Luna)"
        tc_springcereals_sos = f"{tc_springcereals_sos_day} (Ziua) din {tc_springcereals_sos_month} (Luna)"
        tc_springcereals_eos = f"{tc_springcereals_eos_day} (Ziua) din {tc_springcereals_eos_month} (Luna)"

        return {
            "aez_id": aez_id,
            "aez_groupid": aez_groupid,
            "tc_annual_sos": tc_annual_sos,
            "tc_annual_eos": tc_annual_eos,
            "tc_wintercereals_sos": tc_wintercereals_sos,
            "tc_wintercereals_eos": tc_wintercereals_eos,
            "tc_springcereals_sos": tc_springcereals_sos,
            "tc_springcereals_eos": tc_springcereals_eos
        }
    else:
        return None



# Funcția pentru extragerea informațiilor și afișarea acestora la desenarea poligonului
def on_draw(self, action, geo_json):
    if action == 'created':
        coords = geo_json.get('geometry').get('coordinates')
        polygon_coords = ee.Geometry.Polygon(coords)

        # Extrage informații SMAP
        smap_info = get_smap_info_for_region(polygon_coords)
        if smap_info:
            print("Informații SMAP:")
            print(f"Umiditate sol de suprafață: {smap_info['surface_soil_moisture']}% cm")
            print(f"Umiditate sol zona rădăcinii: {smap_info['root_zone_soil_moisture']}% cm")
            print(f"Temperaturi: {smap_info['surface_zone_temp_celsius']} C")
            print(f"Flux evapotranspirație teren: {smap_info['land_zone_evapotranspiration_flux']} kg/m^2/s")
            print(f"Flux infiltrare apă în sol: {smap_info['soil_zone_water_infiltration_flux']} kg/m^2/s")

        # Extrage informații ESA WorldCereal
        esa_info = get_esa_info_for_region(polygon_coords)
        if esa_info:
            print("\nInformații ESA WorldCereal:")
            print(f"AEZ ID (ID zona agro-ecologică): {esa_info['aez_id']}")
            print(f"AEZ Group ID (ID grup zona agro-ecologică): {esa_info['aez_groupid']}")
            print(f"TC Annual SOS (Ziua de start a sezonului anual de cultură): {esa_info['tc_annual_sos']}")
            print(f"TC Annual EOS (Ziua de sfârșit a sezonului anual de cultură): {esa_info['tc_annual_eos']}")
            print(f"TC Wintercereals SOS (Ziua de start a sezonului de iernare a cerealelor): {esa_info['tc_wintercereals_sos']}")
            print(f"TC Wintercereals EOS (Ziua de sfârșit a sezonului de iernare a cerealelor): {esa_info['tc_wintercereals_eos']}")
            print(f"TC Springcereals SOS (Ziua de start a sezonului de primăvară a cerealelor): {esa_info['tc_springcereals_sos']}")
            print(f"TC Springcereals EOS (Ziua de sfârșit a sezonului de primăvară a cerealelor): {esa_info['tc_springcereals_eos']}")
        else:
            print("Nu există date disponibile pentru zona selectată.")


# Crearea unei hărți
m = Map(center=(0, 0), zoom=2)

# Adăugarea controlului pentru desenarea poligonului pe hartă
draw_control = DrawControl()
draw_control.on_draw(on_draw)
m.add_control(draw_control)

# Afișarea hărții
m

EEException: ignored

MODEL 02

In [6]:
import ee
from ipyleaflet import Map, DrawControl

# Autentificare și inițializare Earth Engine
ee.Initialize()

# Definirea unei funcții pentru extragerea informațiilor SMAP pentru regiunea specificată
def get_smap_info_for_region(geometry):
    smap_bands = ee.ImageCollection("NASA/SMAP/SPL4SMGP/007")
    smap_bands_in_region = smap_bands.filterBounds(geometry)
    first_smap_bands = smap_bands_in_region.first()

    smap_data = first_smap_bands.select(['sm_profile_pctl', 'sm_rootzone_pctl','surface_temp', 'land_evapotranspiration_flux', 'soil_water_infiltration_flux']).reduceRegion(
        reducer=ee.Reducer.mean(), geometry=geometry, scale=9000).getInfo()

    surface_soil_moisture = smap_data['sm_profile_pctl']
    root_zone_soil_moisture = smap_data['sm_rootzone_pctl']
    land_zone_evapotranspiration_flux = smap_data['land_evapotranspiration_flux']
    soil_zone_water_infiltration_flux = smap_data['soil_water_infiltration_flux']

    surface_zone_temp_kelvin = smap_data['surface_temp']
    surface_zone_temp_celsius = surface_zone_temp_kelvin - 273.15

    return {
        "surface_soil_moisture": surface_soil_moisture,
        "root_zone_soil_moisture": root_zone_soil_moisture,
        "surface_zone_temp_celsius": surface_zone_temp_celsius,
        "land_zone_evapotranspiration_flux": land_zone_evapotranspiration_flux,
        "soil_zone_water_infiltration_flux": soil_zone_water_infiltration_flux
    }

# Definirea unei funcții pentru extragerea informațiilor din setul de date ESA WorldCereal pentru regiunea specificată
def get_esa_info_for_region(geometry):
    aez_dataset = ee.FeatureCollection("ESA/WorldCereal/AEZ/v100")
    features = aez_dataset.filterBounds(geometry).getInfo()['features']

    if len(features) > 0:
        properties = features[0]['properties']
        aez_id = properties['aez_id']
        aez_groupid = properties['aez_groupid']
        tc_annual_sos = properties['tc-annual_sos']
        tc_annual_eos = properties['tc-annual_eos']
        tc_wintercereals_sos = properties['tc-wintercereals_sos']
        tc_wintercereals_eos = properties['tc-wintercereals_eos']
        tc_springcereals_sos = properties['tc-springcereals_sos']
        tc_springcereals_eos = properties['tc-springcereals_eos']

        return {
            "aez_id": aez_id,
            "aez_groupid": aez_groupid,
            "tc_annual_sos": tc_annual_sos,
            "tc_annual_eos": tc_annual_eos,
            "tc_wintercereals_sos": tc_wintercereals_sos,
            "tc_wintercereals_eos": tc_wintercereals_eos,
            "tc_springcereals_sos": tc_springcereals_sos,
            "tc_springcereals_eos": tc_springcereals_eos
        }
    else:
        return None

# Funcția pentru extragerea informațiilor și afișarea acestora la desenarea poligonului
def on_draw(self, action, geo_json):
    if action == 'created':
        coords = geo_json.get('geometry').get('coordinates')
        polygon_coords = ee.Geometry.Polygon(coords)

        # Extrage informații SMAP
        smap_info = get_smap_info_for_region(polygon_coords)
        if smap_info:
            print("Informații SMAP:")
            print(f"Umiditate sol de suprafață: {smap_info['surface_soil_moisture']}% ")
            print(f"Umiditate sol zona rădăcinii: {smap_info['root_zone_soil_moisture']}% ")
            print(f"Temperaturi: {smap_info['surface_zone_temp_celsius']} C")
            print(f"Flux evapotranspirație teren: {smap_info['land_zone_evapotranspiration_flux']} kg/m^2/s")
            print(f"Flux infiltrare apă în sol: {smap_info['soil_zone_water_infiltration_flux']} kg/m^2/s")

        # Extrage informații ESA WorldCereal
        esa_info = get_esa_info_for_region(polygon_coords)
        if esa_info:
            print("\nInformații ESA WorldCereal:")
            print(f"AEZ ID (ID zona agro-ecologică): {esa_info['aez_id']}")
            print(f"AEZ Group ID (ID grup zona agro-ecologică): {esa_info['aez_groupid']}")
            print(f"TC Annual SOS (Ziua de start a sezonului anual de cultură): {esa_info['tc_annual_sos']}")
            print(f"TC Annual EOS (Ziua de sfârșit a sezonului anual de cultură): {esa_info['tc_annual_eos']}")
            print(f"TC Wintercereals SOS (Ziua de start a sezonului de iernare a cerealelor): {esa_info['tc_wintercereals_sos']}")
            print(f"TC TC Wintercereals EOS (Ziua de sfârșit a sezonului de iernare a cerealelor): {esa_info['tc_wintercereals_eos']}")
            print(f"TC Springcereals SOS (Ziua de start a sezonului de primăvară a cerealelor): {esa_info['tc_springcereals_sos']}")
            print(f"TC Springcereals EOS (Ziua de sfârșit a sezonului de primăvară a cerealelor): {esa_info['tc_springcereals_eos']}")
        else:
            print("Nu există date disponibile pentru zona selectată.")

# Crearea unei hărți
m = Map(center=(0, 0), zoom=2)

# Adăugarea controlului pentru desenarea poligonului pe hartă
draw_control = DrawControl()
draw_control.on_draw(on_draw)
m.add_control(draw_control)

# Afișarea hărții
m


Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

Informații SMAP:
Umiditate sol de suprafață: 60.50721740722656% 
Umiditate sol zona rădăcinii: 60.50721740722656% 
Temperaturi: 3.9493652343750227 C
Flux evapotranspirație teren: 1.4232064131647348e-06 kg/m^2/s
Flux infiltrare apă în sol: 1.589651219546795e-07 kg/m^2/s

Informații ESA WorldCereal:
AEZ ID (ID zona agro-ecologică): 22190
AEZ Group ID (ID grup zona agro-ecologică): 22000
TC Annual SOS (Ziua de start a sezonului anual de cultură): 305
TC Annual EOS (Ziua de sfârșit a sezonului anual de cultură): 304
TC Wintercereals SOS (Ziua de start a sezonului de iernare a cerealelor): 44
TC TC Wintercereals EOS (Ziua de sfârșit a sezonului de iernare a cerealelor): 223
TC Springcereals SOS (Ziua de start a sezonului de primăvară a cerealelor): 107
TC Springcereals EOS (Ziua de sfârșit a sezonului de primăvară a cerealelor): 304
